# Edit Universe
We will expand primary cluster in existing universe by increasing numer of nodes in the primary cluster

### Setup

First, import the required packages.

Next, specify some important variables:
* `platform_address`: The address of the Yugabyte Platform API
* `platform_api_key`: The API key used to authenticate with the Platform API

Finally, open a HTTP connection to the Yugabyte Platform API.

In [9]:
import os
import http.client
import json
from pprint import pprint

platform_address = os.getenv('API_BASE_URL', "portal.dev.yugabyte.com")
platform_api_key = os.getenv('YB_API_KEY')

conn = http.client.HTTPConnection(f"{platform_address}")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{platform_api_key}"
}

### Get Session Info

Make an API call to session endpoint to determine customer UUID.

In [10]:
conn.request("GET", "/api/v1/session_info", headers=headers)

res = conn.getresponse()
data = res.read()
session_info = json.loads(data)
customer_uuid = session_info['customerUUID']

print('Customer UUID:\n%s' % customer_uuid)

Customer UUID:
11d78d93-1381-4d1d-8393-ba76f47ba7a6


### GET Universe By Name
The way to do it is list universes with `name` query parameter:

In [11]:
universe_uuid = "303cf5ac-a6f5-492b-b68c-7d4d9633c4cc" # or ybTask.resourceUUID
url = f"/api/v1/customers/{customer_uuid}/universes?name=my-gcp-universe"
conn.request("GET", url, headers=headers)
res = conn.getresponse()
the_universe = json.load(res)[0]
# pprint(the_universe['universeDetails'])

### Update Universe

Make API call to update the newly created universe.

In [7]:
new_universe = the_universe['universeDetails']
new_universe['clusters'][0]['userIntent']['numNodes'] = new_universe['clusters'][0]['userIntent']['numNodes'] + 2
url = f"/api/v1/customers/{customer_uuid}/universes/{the_universe['universeUUID']}/clusters/primary"
conn.request("PUT", url, json.dumps(new_universe), headers)
res = conn.getresponse()
print(res.status)
pprint(json.load(res))

200
{'resourceUUID': '470840d0-fe78-4552-924e-ff61b1d16c7d',
 'taskUUID': '5030a5f9-7138-40cc-988e-0894694df673'}
